BÁO CÁO TÌM HIỂU THƯ VIỆN DOCTR (DOCUMENT TEXT RECOGNITION)

Giới thiệu chung
docTR (Document Text Recognition) là thư viện mã nguồn mở do Mindee phát triển, được xây dựng trên nền PyTorch và TensorFlow, dùng cho bài toán OCR (Optical Character Recognition) – trích xuất văn bản từ ảnh hoặc tài liệu PDF bằng kỹ thuật Deep Learning.
docTR cung cấp pipeline end-to-end gồm hai giai đoạn chính:
(1) Text Detection – Xác định vùng chứa văn bản trong ảnh.
(2) Text Recognition – Nhận dạng nội dung chữ trong từng vùng được phát hiện.
Mục tiêu của docTR là thay thế các công cụ OCR truyền thống như Tesseract bằng giải pháp học sâu có độ chính xác cao và linh hoạt hơn.

Kiến trúc tổng quan
Quy trình xử lý tài liệu trong docTR:
Ảnh tài liệu → Phát hiện văn bản → Cắt vùng chữ → Nhận dạng ký tự → Xuất kết quả (text, JSON hoặc ảnh chú thích).

Các mô hình được hỗ trợ:

Detection: DBNet, LinkNet, CTPN.

Recognition: CRNN, SAR, VisionTransformer.
Mỗi mô hình đều có phiên bản pretrained để sử dụng ngay.

Cách cài đặt và sử dụng (Google Colab)
!pip install python-doctr torch torchvision matplotlib pillow
from doctr.models import ocr_predictor
from doctr.io import DocumentFile
model = ocr_predictor(pretrained=True)
doc = DocumentFile.from_images("sample.jpg")
result = model(doc)
print(result.render())

Hiển thị kết quả:
import matplotlib.pyplot as plt
fig = result.show(doc)
plt.show()

Đánh giá thực nghiệm
Ưu điểm:

Nhận dạng chính xác cao với tiếng Anh và ký tự Latin.

Giữ được bố cục và vị trí văn bản trong ảnh.

Dễ sử dụng, hỗ trợ GPU, có thể mở rộng.
Hạn chế:

Chưa tối ưu cho tiếng Việt, đặc biệt chữ có dấu hoặc font viết tay.

Dữ liệu huấn luyện chủ yếu là tiếng Anh.

Không có sẵn model tiếng Việt.
Ví dụ:
“Cộng hòa xã hội chủ nghĩa Việt Nam” → “Cong hoa xa hoi chu nghia Viet Nam”
“Giấy chứng minh nhân dân” → “Giay chung minh nhan dan”
=> Mất dấu do mô hình recognition chưa học đặc trưng tiếng Việt.

Hướng phát triển và tinh chỉnh cho tiếng Việt
Mục tiêu: Cải thiện độ chính xác nhận dạng tiếng Việt (đọc đúng dấu, font chữ Việt).

Hướng đi đề xuất:
(1) Fine-tune mô hình recognition

Giữ nguyên detection (DBNet pretrained).

Fine-tune recognition (CRNN, SAR, Transformer) trên dataset tiếng Việt có nhãn text.

Dataset có thể dùng: Vietnamese Scene Text (UIT-ViText) hoặc tự tạo từ hóa đơn, giấy tờ.

Chuẩn hóa annotation theo format JSON của docTR.

(2) Mở rộng bảng ký tự

Cập nhật charset trong recognition để bao gồm đầy đủ dấu tiếng Việt: ă, â, ê, ô, ơ, ư, đ, á, à, ả, ã, ạ,...

Giúp mô hình học mapping chính xác giữa hình dạng và ký tự có dấu.

(3) Data augmentation

Tăng cường dữ liệu bằng xoay, làm mờ, thay đổi độ sáng, thêm nhiễu.

Giúp mô hình ổn định hơn khi gặp ảnh thực tế.

(4) Xử lý sau (post-processing)

Dùng từ điển hoặc mô hình ngôn ngữ tiếng Việt để hiệu chỉnh lỗi chính tả, ví dụ “Cong hoa” → “Cộng hòa”.

Kết luận
docTR là công cụ mạnh mẽ, hiện đại cho OCR bằng học sâu, dễ sử dụng và triển khai.
Tuy nhiên, để ứng dụng hiệu quả cho tiếng Việt cần:

Fine-tune mô hình recognition,

Cập nhật bảng ký tự có dấu,

Dùng dataset tiếng Việt phù hợp.
Tinh chỉnh này khả thi nhờ kiến trúc mô-đun của docTR và có thể giúp tăng độ chính xác OCR trong các ứng dụng như số hóa hồ sơ, nhận dạng giấy tờ, trích xuất thông tin hành chính, hóa đơn điện tử.

In [3]:
!pip install python-doctr torch torchvision matplotlib pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 42.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=ec434b431286b0a6826ac9924fa0a3ee6623848e86362647d20bf0c15389cdc5
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e

In [4]:
from doctr.models import ocr_predictor
from doctr.io import DocumentFile

# tạo model OCR có sẵn (end-to-end)
model = ocr_predictor(pretrained=True)

  0%|          | 0/65814772 [00:00<?, ?it/s]

  0%|          | 0/63303144 [00:00<?, ?it/s]

In [6]:
from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())[0]

# đọc ảnh thành DocumentFile
doc = DocumentFile.from_images(filename)

# chạy OCR
result = model(doc)

# in ra kết quả text
print(result.render())

Saving 553655723_1123929623268638_7620765683896878252_n.jpg to 553655723_1123929623268638_7620765683896878252_n.jpg
27 A Hoany Viet Phudng
Quan Tar Bink
TP do Chi minh


FINE-TUNE MÔ HÌNH DOCTR CHO TIẾNG VIỆT

Mục tiêu
Thư viện docTR (Document Text Recognition) có sẵn các mô hình OCR pretrained cho tiếng Anh và các ký tự Latin. Tuy nhiên, các mô hình này chưa đọc tốt tiếng Việt có dấu. Mục tiêu của việc fine-tune là tinh chỉnh phần Text Recognition để mô hình nhận dạng đúng các ký tự tiếng Việt, trong khi vẫn giữ nguyên phần phát hiện chữ (Text Detection).

Cấu trúc mô hình docTR
Hệ thống OCR trong docTR gồm hai phần chính:
(1) Text Detection (ví dụ DBNet, LinkNet) dùng để phát hiện vùng chứa chữ trong ảnh.
(2) Text Recognition (ví dụ CRNN, SAR, VisionTransformer) dùng để nhận dạng nội dung chữ trong từng vùng đó.
Trong giai đoạn fine-tune, chỉ cần huấn luyện lại phần Recognition, vì phần Detection không phụ thuộc ngôn ngữ.

Dữ liệu huấn luyện
Đầu vào cần là các ảnh chữ hoặc dòng văn bản, cùng với nhãn text tương ứng.
Cấu trúc thư mục dữ liệu như sau:
data/
├── train/
│ ├── img1.jpg
│ ├── img2.jpg
│ └── labels.json
└── val/
├── img3.jpg
├── img4.jpg
└── labels.json

File labels.json lưu thông tin cặp ảnh - text, ví dụ:
{"annotations": [{"image": "img1.jpg", "text": "Cộng hòa xã hội chủ nghĩa Việt Nam"}, {"image": "img2.jpg", "text": "Giấy chứng minh nhân dân"}]}

Bảng ký tự tiếng Việt
docTR mặc định chỉ có bảng ký tự Latin. Để huấn luyện cho tiếng Việt, cần mở rộng charset để bao gồm các ký tự có dấu:
0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZàáạảãăằắẳẵặâầấẩẫậèéẹẻẽêềếểễệìíịỉĩòóọỏõôồốổỗộơờớởỡợùúụủũưừứửữựỳýỵỷỹđ và khoảng trắng.

Quy trình fine-tune
Bước 1: Nạp dataset tiếng Việt đã chuẩn bị, định nghĩa hàm Resize để chuẩn hóa ảnh (ví dụ 32x128 pixel).
Bước 2: Tạo mô hình recognition mới, ví dụ crnn_vgg16_bn, với vocab=charset tiếng Việt.
Bước 3: Cấu hình Trainer trong docTR, chỉ định optimizer (Adam), learning rate (1e-4), số epoch (ví dụ 50), hàm loss (CTC) và metric (TextMatch).
Bước 4: Tiến hành huấn luyện bằng trainer.fit(). Sau khi hoàn tất, lưu trọng số model (ví dụ crnn_vi.pth).
Bước 5: Load lại mô hình đã huấn luyện và kiểm tra bằng ảnh thử nghiệm.

Lưu ý khi huấn luyện

Nếu dữ liệu nhỏ, nên freeze backbone CNN và chỉ train phần LSTM + Linear.

Dùng data augmentation (xoay ±5°, thay nền, thêm nhiễu) để tăng khả năng tổng quát.

Có thể tăng kích thước ảnh (W=256 hoặc 512) nếu text dài.

Giảm learning rate hoặc dùng dropout khi bị overfit.

Nếu cần mô hình mạnh hơn, có thể thay CRNN bằng SAR hoặc Transformer.

Đánh giá và ứng dụng
Sau khi fine-tune, mô hình có thể đọc chính xác các ký tự tiếng Việt có dấu và giữ nguyên bố cục trong văn bản. Hệ thống có thể được áp dụng cho các bài toán như số hóa hồ sơ, nhận dạng hóa đơn, trích xuất thông tin hành chính, hoặc tài liệu tiếng Việt nói chung.

Kết luận
Fine-tune docTR cho tiếng Việt giúp cải thiện đáng kể độ chính xác của OCR so với mô hình mặc định. Việc mở rộng bảng ký tự và huấn luyện lại Recognition model trên dữ liệu tiếng Việt là hướng đi thiết thực, giúp thư viện docTR thích ứng với ngôn ngữ có dấu phức tạp như tiếng Việt.